Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# CSE 204 Lab 14: Kernel methods

<img src="https://raw.githubusercontent.com/adimajo/CSE204-2021/master/data/logo.jpg" style="float: left; width: 15%" />

[CSE204-2021](https://moodle.polytechnique.fr/course/view.php?id=12838) Lab session #14

Jérémie Decock

## Introduction

In this lab, you will implement two kernel methods: Kernel k-Means and Kernel PCA.
While libraries such as scikit-learn implement these algorithms, they are simple enough for you to implement with numpy alone.
Before starting, import the required packages.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from random import randrange
import pandas as pd
import itertools
import random

## Part 1: Kernel methods

### Introduction

Some machine learning algorithms we have seen so far only work for "linear problems".

For instance:
- with linear regression ([`lab_session_02`](https://adimajo.github.io/CSE204-2021/lab_session_02/lab_session_02.html)) we fit a hyperplane to predict unknown values, but the method can't effectively predict $f(\boldsymbol{x})$ if the (unknown) $f$ function is non-linear with respect to $\boldsymbol{x}$;
- with linear classification methods, e.g. logistic regression ([`lab_session_05`](https://adimajo.github.io/CSE204-2021/lab_session_05/lab_session_05.html)), the decision boundary is a hyperplane;
- the Principal Component Analysis (PCA) algorithm for feature extraction ([`lab_session_10`](https://adimajo.github.io/CSE204-2021/lab_session_10/lab_session_10.html)) can only find "linear directions" (meaning linear combinations of the original features) in the data;
- in the k-means algorithm presented last week for clustering tasks ([`lab_session_12`](https://adimajo.github.io/CSE204-2021/lab_session_12/lab_session_12.html)), the separating boundary between clusters is linear (i.e. k-Means can only detect convex clusters).

As we have seen in [`lab_session_04`](https://adimajo.github.io/CSE204-2021/lab_session_04/lab_session_04.html), where we introduced polynomial regression, we can extend all these methods to "nonlinear problems" by applying a feature mapping $\phi$ on the data.
This function maps input data $\boldsymbol{x}_i \in \mathbb{R}^d$ into another space (the *feature space*) $\phi(\boldsymbol{x}_i) \in \mathbb{R}^{\hat{d}}$ where the "linear algorithm" is actually effective.
Here we denote by $\mathcal{X}$ the input space and by $\mathcal{H}$ the feature space.

To be effective, the feature space is defined as a very high dimension space in many practical problems (usually $\hat{d} \gg d$, contrary to what we did in [`lab_session_10`](https://adimajo.github.io/CSE204-2021/lab_session_10/lab_session_10.html) and [`lab_session_11`](https://adimajo.github.io/CSE204-2021/lab_session_11/lab_session_11.html) with PCA and autoencoders respectively, where the objectives were visualization and compression - $d$ was already "big" - no pun intended).
Thus often, projections and computations in such a space are tedious and require a lot of computational power.

So here comes the *Kernel trick*.
The basic idea is to project data $\boldsymbol{x}_i$ in some high dimension feature space $\mathcal{H}$ and apply an adapted version of the "linear algorithms" in this space without explicitly computing the projection $\phi(\boldsymbol{x}_i)$.

Using the kernel trick, we often don't even know what the feature map $\phi$ actually is!
Instead, computations are made implicitly from $\mathcal{X}$ through a carefully chosen function $K: \mathcal{X} \times \mathcal{X} \rightarrow \mathbb{R}$ which computes some kind of "similarity" between two points of $\mathcal{X}$.

This similarity function $K$ named *kernel* is chosen such that it represents a dot product in the high-dimensional feature space.

Thus, to be a valid kernel, $K$ have to be defined such that there is a $\phi$ function for which the following holds: $K(\boldsymbol{x}_i, \boldsymbol{x}_j) = \phi(\boldsymbol{x}_i)^{\top} \phi(\boldsymbol{x}_j)$ for all $\boldsymbol{x}_i, \boldsymbol{x}_j \in \mathcal{X}$.

The *Kernel matrix* $\mathbf{K}$ is a $n \times n$ matrix containing the pairwise similarity values between points in the dataset $\mathbf{D} = (\boldsymbol{x}_i)_1^n \subset \mathcal{X}$:
$$
\mathbf{K} =
\begin{pmatrix}
K(\boldsymbol{x}_1, \boldsymbol{x}_1) & K(\boldsymbol{x}_1, \boldsymbol{x}_2) & \cdots & K(\boldsymbol{x}_1, \boldsymbol{x}_n)\\
K(\boldsymbol{x}_2, \boldsymbol{x}_1) & K(\boldsymbol{x}_2, \boldsymbol{x}_2) & \cdots & K(\boldsymbol{x}_2, \boldsymbol{x}_n)\\
\vdots                        & \vdots                        & \ddots & \vdots\\
K(\boldsymbol{x}_n, \boldsymbol{x}_1) & K(\boldsymbol{x}_n, \boldsymbol{x}_2) & \cdots & K(\boldsymbol{x}_n, \boldsymbol{x}_n)
\end{pmatrix}
$$

Eventually, the kernel function $K$ allows us to compute a dot product in $\mathcal{H}$ without explicitly knowing (and calculating) $\phi(\boldsymbol{x})$. The idea is then to find some machine learning algorithms that can be rewritten such that they only require the dot product $\phi(\boldsymbol{x}_i)^{\top} \phi(\boldsymbol{x}_j)$ in the feature space, *i.e.*, algorithms where $\phi$ only appears in the form of a dot product $\phi(\boldsymbol{x}_i)^{\top} \phi(\boldsymbol{x}_j)$ (so that eventually all computations in $\mathcal{H}$ can be performed exclusively using $K$).

PCA, k-Means and ridge regression are such compatible (*kernelizable*) algorithms: that is approximately what we did for spectral clustering [in the last lab](https://adimajo.github.io/CSE204-2021/lab_session_13/lab_session_13.html) where the kernel was given by the $k$ nearest neighbors.

$||\cdot||$ will denote the L2 norm in what follows.

We will now present some classical kernels. Let's start with the simplest one: the linear kernel.

#### The linear kernel

The linear kernel is defined as: $K(\boldsymbol{x}, \boldsymbol{y}) = \boldsymbol{x}^{\top} \boldsymbol{y}$ <br>
with $\boldsymbol{x}, \boldsymbol{y} \in \mathcal{X}$.

Thus the feature mapping associated to this kernel is the identity mapping $\phi(\boldsymbol{x}) \mapsto \boldsymbol{x}$.

For instance, for two points $\boldsymbol{x} = \pmatrix{1 \\ 2}$ $\boldsymbol{y} = \pmatrix{3 \\ 1}$ in $\mathbf{D}$, we have:
$$K(\boldsymbol{x}, \boldsymbol{y}) = 1 \times 3 + 2 \times 1 = 5$$

**Note**: The linear kernel is not really the most useful in practice; "linear algorithms" won't work better on "nonlinear problems" using it. But it's a useful kernel to check some theoretical properties.

#### The polynomial kernel

The *inhomogeneous polynomial kernel* of degree $q$ is defined as: $K_q(\boldsymbol{x}, \boldsymbol{y}) = \left( c + \boldsymbol{x}^{\top} \boldsymbol{y} \right)^q$, <br>
with $\boldsymbol{x}, \boldsymbol{y} \in \mathcal{X}$ and some constant $c \geq 0$.
$K$ is an *homogeneous polynomial kernel* when $c=0$.

The feature mapping $\phi: \mathbb{R}^d \rightarrow \mathbb{R}^\hat{d}$ associated to the polynomial kernel is:
$$
\phi(\boldsymbol{x}) = \pmatrix{\cdots & \sqrt{\pmatrix{q \\ \boldsymbol{n}} c^{n_0}} \prod_{k=1}^{d} x_k^{n_k} & \cdots} 
$$
where the variable $\boldsymbol{n} = (n_0, \dots, n_d)$ is the vector of non-negative integers such that $\sum_{i=0}^d n_i = q$,
and where $\pmatrix{q \\ \boldsymbol{n}}$ denotes the multinomial coefficient:

$$
\pmatrix{q \\ \boldsymbol{n}} = \pmatrix{q \\ n_0, n_1, \dots, n_d} = \frac{q!}{n_0! n_1! \dots n_d!}
$$

The dimensionality of the feature space is $\hat{d} = \pmatrix{d + q \\ q}$.

For instance, for two points $\boldsymbol{x} = \pmatrix{1 \\ 2}$, $\boldsymbol{y} = \pmatrix{3 \\ 1}$ in $\mathbf{D}$ and for the *homogeneous quadratic kernel* ($q=2$ and $c=0$) we have:
$$K_2(\boldsymbol{x}, \boldsymbol{y}) = (1 \times 3 + 2 \times 1)^2 = 25$$

#### The Gaussian kernel

The Gaussian kernel is defined as:
$$
K(\boldsymbol{x}, \boldsymbol{y}) = \exp\left( \frac{- ||\boldsymbol{x} - \boldsymbol{y}||^2}{2 \sigma^2} \right)
$$

where $||\boldsymbol{x} - \boldsymbol{y}||_2^2$ is the Euclidean distance between $\boldsymbol{x}$ and $\boldsymbol{y}$
and where $\sigma > 0$ is a given "spread", or "bandwidth" parameter that plays the same role as the standard deviation in a normal density function.

Note that $K(\boldsymbol{x}, \boldsymbol{x}) = 1$ and that the kernel value is inversely related to the distance between the two points $\boldsymbol{x}$ and $\boldsymbol{y}$.

The feature mapping $\phi: \mathbb{R}^d \rightarrow \mathbb{R}^{\infty}$ associated to the Gaussian kernel **has infinite dimensions**, thus we cannot explicitly transform $\boldsymbol{x}$ into $\phi(\boldsymbol{x})$ (see [What about inifinite dimensions?](https://medium.com/analytics-vidhya/the-kernel-trick-and-infinite-dimensions-7ecd91cee6ef)).
However computing the Gaussian kernel $K(\boldsymbol{x}, \boldsymbol{y})$ is straightforward.

For instance, for two points $\boldsymbol{x} = \pmatrix{1 \\ 2}$, $\boldsymbol{y} = \pmatrix{3 \\ 1}$ in $\mathbf{D}$ and the *Gaussian kernel* (with $\sigma=1$) we have
$K(\boldsymbol{x}, \boldsymbol{y}) \approx 0.082$.

### Exercise 1

Implement in Python the following kernels:
- Linear kernel

**Remark**: We use classes to implement kernels as [*functors*](https://en.wikipedia.org/wiki/Function_object#In_Python) (parametric functions i.e. classes with a defined `__call__` method), see [`lab_session_06`](https://adimajo.github.io/CSE204-2021/lab_session_06/lab_session_06.html) and [`lab_session_07`](https://adimajo.github.io/CSE204-2021/lab_session_07/lab_session_07.html).

In [ ]:
class LinearKernel:
    """
    The linear kernel is defined as:
    
    ..math::
        K(\mathbf{x}, \mathbf{y}) = \mathbf{x}^{\top} \mathbf{y}

    The feature mapping associated to this kernel is the identity mapping :math:`\phi(\mathbf{x}) \mapsto \mathbf{x}`.
    
    Parameters
    ----------
    x1 : numpy.ndarray
        a point of the input space (1D numpy array)
    x2 : numpy.ndarray
        a point of the input space (1D numpy array)
    
    Return
    ------
    k : float
        the similarity K(x1, x2)
    """
    def __str__(self):
        """
        __str__ is called whenever you call str(object_of_that_class)
        """
        return "Linear"

    def __call__(self, x1: np.ndarray, x2: np.ndarray):
        """
        Compute the Gaussian Kernel

        Parameters
        ----------
        x1 : numpy.ndarray
            a point of the input space (1D numpy array)
        x2 : numpy.ndarray
            a point of the input space (1D numpy array)

        Return
        ------
        k : float
            the similarity K(x1, x2)
        """

        # YOUR CODE HERE
        raise NotImplementedError()

        return k

- Polynomial kernel

In [ ]:
class PolynomialKernel:
    """
    The *inhomogeneous polynomial kernel* of degree $q$ is defined as:

    ..math::
        K_q(\mathbf{x}, \mathbf{y}) = \left( c + \mathbf{x}^{\top} \mathbf{y} \right)^q

    with :math:`\mathbf{x}, \mathbf{y} \in \mathcal{X}` and some constant :math:`c \geq 0`.
    :math:`K` is an *homogeneous polynomial kernel* when :math:`c=0`.

    Attributes
    ----------
    q : float
        the degree of the polynomial kernel
    c : float
        the polynomial constant
    """
    def __init__(self, q: int = 2, c: int = 0):
        """
        Initialize the inhomogeneous polynomial kernel with values for q (degree) and c (constant) - see above
        """
        self.q = q
        self.c = c

    def __str__(self):
        return "Polynomial"

    def __call__(self, x1: np.ndarray, x2: np.ndarray):
        """
        Compute the polynomial Kernel

        Parameters
        ----------
        x1 : numpy.ndarray
            a point of the input space (1D numpy array)
        x2 : numpy.ndarray
            a point of the input space (1D numpy array)

        Return
        ------
        k : float
            the similarity K(x1, x2)
        """

        # YOUR CODE HERE
        raise NotImplementedError()

        return k

- Gaussian kernel 

In [ ]:
class GaussianKernel:
    """
    The Gaussian kernel is defined as:

    ..math::
        K(\boldsymbol{x}, \boldsymbol{y}) = \exp \left( \frac{- ||\boldsymbol{x} - \boldsymbol{y}||_2^2}{2 \sigma^2} \right)

    where :math:`||\boldsymbol{x} - \boldsymbol{y}||_2^2` is the Euclidean distance between :math:`x_1` and :math:`x_2`,
    and where :math:`\sigma > 0` is a given "spread" parameter that plays the same role as the standard deviation in a normal density function.

    Attributes
    ----------
    sigma : float
        spread or bandwidth parameter
    """
    
    def __init__(self, sigma: float = 1.0):
        self.sigma = sigma

    def __str__(self):
        return "Gaussian"

    def __call__(self, x1: np.ndarray, x2: np.ndarray):
        """Compute the Gaussian Kernel

        Parameters
        ----------
        x1 : ndarray
            a point of the input space (1D numpy array)
        x2 : ndarray
            a point of the input space (1D numpy array)

        Return
        ------
        k : float
            the similarity K(x1, x2)
        """

        # YOUR CODE HERE
        raise NotImplementedError()

        return k

In [ ]:
x1 = np.array([1, 2])
x2 = np.array([3, 1])

kernel = LinearKernel()
print(kernel(x1, x2))
assert kernel(x1, x2) == 5

kernel = PolynomialKernel(q=2, c=0)
print(kernel(x1, x2))
assert kernel(x1, x2) == 25

kernel = GaussianKernel(sigma=1)
print(kernel(x1, x2))
assert round(kernel(x1, x2), 3) == 0.082

### Exercise 2

Implement in Python the following function to make a kernel matrix from a given kernel function $K$ and a given dataset $\mathbf{D}$.

**Reminder**: in Python, a function is an object that can be passed as an argument to a function like any variable (cf. the implementation of `linear_kernel_matrix`, `polynomial_kernel_matrix` and `gaussian_kernel_matrix` hereafter).

*Hints*: you can create an empty square matrix with `np.zeros`, `np.ones`, `np.full`... You can do a double for-loop to populate it.

**Bonus**: explore [this webpage](https://numpy.org/doc/stable/reference/generated/numpy.ufunc.outer.html) to vectorize your implementation.

In [ ]:
def make_kernel_matrix(data: np.ndarray, kernel_function) -> np.ndarray:
    """
    Make a Kernel matrix.

    Parameters
    ----------
    data : ndarray
        the dataset (2D numpy array) with examples in rows and dimensions in columns
    kernel_function : function
        the kernel function used to make the kernel matrix

    Return
    ------
    K : ndarray
        the n x n kernel matrix
    """

    # YOUR CODE HERE
    raise NotImplementedError()

    return K

In [ ]:
# The following functions are provided here for convenience with default parameters

def linear_kernel_matrix(data):
    return make_kernel_matrix(data, LinearKernel())

def polynomial_kernel_matrix(data):
    return make_kernel_matrix(data, PolynomialKernel())

def gaussian_kernel_matrix(data):
    return make_kernel_matrix(data, GaussianKernel())

In [ ]:
# Check on a dataset of 3 points in 2 dimensions
data = np.array([[1., 2.],
                 [3., 1.],
                 [2., 3.]])

print(linear_kernel_matrix(data), "\n")
print(polynomial_kernel_matrix(data), "\n")
print(gaussian_kernel_matrix(data), "\n")

### Exercise 3: Basic kernel operations in feature space

In this exercise, we will look at some of the basic operations that can be performed in the feature space solely via kernels (i.e. without computing $\phi(\boldsymbol{x})$).
These are the standard building blocks that are used to *kernelize* algorithms like k-Means, PCA, ...

We will see that we can get the norm of a point in the feature space, the distance between two points, the mean of the training dataset and the variance respectively. We will proceed by centering, and optionnally normalize points in the feature space.

#### Norm of a point

**Question 1:**
Show that we can compute the norm of a point $\phi(\boldsymbol{x})$ in the feature space as follows:
$$
\left\lVert \phi(\boldsymbol{x}) \right\rVert = \sqrt{K(\boldsymbol{x}, \boldsymbol{x})}
$$

YOUR ANSWER HERE

#### Distance between points

**Question 2:**
Show that the distance between two points $\phi(\boldsymbol{x}_i)$ and $\phi(\boldsymbol{x}_j)$ in the feature space can be computed as:
$$
\left\lVert \phi(\boldsymbol{x}_i) - \phi(\boldsymbol{x}_j) \right\rVert = \sqrt{K(\boldsymbol{x}_i, \boldsymbol{x}_i) + K(\boldsymbol{x}_j, \boldsymbol{x}_j) - 2 K(\boldsymbol{x}_i, \boldsymbol{x}_j)}
$$

YOUR ANSWER HERE

#### Mean in feature space

**Question 3:**
Show that the squared norm of the mean vector $\boldsymbol{\mu}_{\phi}$ consisting of the mean of each coordinate in the feature space for all training points $\boldsymbol{x}_i$ for $1 \leq i \leq n$ can be computed as:
$$
\left\lVert \boldsymbol{\mu}_{\phi} \right\rVert^2
= \boldsymbol{\mu}_{\phi}^{\top} \boldsymbol{\mu}_{\phi}
= \frac{1}{n^2} \sum_{i=1}^n \sum_{j=1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j)
$$

In other words, show that the squared norm of the mean in feature space is simply the average of the values in the kernel matrix $\mathbf{K}$.

YOUR ANSWER HERE

#### Total variance in feature space

**Question 4:**
Show that the total variance $\sigma_{\phi}^2$ of the training points $\boldsymbol{x}_i$ in the feature space can be computed as:
$$
\sigma_{\phi}^2
= \frac{1}{n} \sum_{i=1}^n \left\lVert \phi(\boldsymbol{x}_i) - \boldsymbol{\mu}_{\phi} \right\rVert^2
= \frac{1}{n} \sum_{i=1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_i) - \frac{1}{n^2} \sum_{i=1}^n \sum_{j=1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j).
$$

In other words, show that the total variance in feature space is simply the difference between the average of the diagonal entries and the average of the entire kernel matrix $\mathbf{K}$.

YOUR ANSWER HERE

#### Centering in feature space

We can center each point in the feature space by subtracting the mean from it: $\tilde{\phi}(\boldsymbol{x}_i) = \phi(\boldsymbol{x}_i) - \boldsymbol{\mu}_{\phi}$.

Remember that we don't have an explicit representation of $\phi(\mathbf{x}_i)$ or $\boldsymbol{\mu}_{\phi}$
but we can still compute the *centered kernel matrix* $\tilde{\mathbf{K}}$, that is the kernel matrix over centered points: $\tilde{\mathbf{K}} = \left\{ \tilde{K}(\boldsymbol{x}_i, \boldsymbol{x}_j) \right\}^n_{i, j=1}$
where each element corresponds to the kernel between centered points $\tilde{K}(\boldsymbol{x}_i, \boldsymbol{x}_j) = \tilde{\phi}(\boldsymbol{x}_i)^{\top} \tilde{\phi}(\boldsymbol{x}_j)$.

**Question 5:**
Show that:
$$
\tilde{K}(\boldsymbol{x}_i, \boldsymbol{x}_j)
= K(\boldsymbol{x}_i, \boldsymbol{x}_j)
- \frac{1}{n} \sum_{k=1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_k)
- \frac{1}{n} \sum_{k=1}^n K(\boldsymbol{x}_j, \boldsymbol{x}_k)
+ \frac{1}{n^2} \sum_{a=1}^n \sum_{b=1}^n K(\boldsymbol{x}_a, \boldsymbol{x}_b)
$$

YOUR ANSWER HERE

**Note**: this can be rewritten in matrix form as:
\begin{align}
\tilde{\mathbf{K}}
& =
  \mathbf{K}
- \frac{1}{n} ~ \mathbf{1}_{n \times n} ~ \mathbf{K}
- \frac{1}{n} ~ \mathbf{K} ~ \mathbf{1}_{n \times n}
+ \frac{1}{n^2} ~ \mathbf{1}_{n \times n} ~ \mathbf{K} ~ \mathbf{1}_{n \times n} \\
& = \left( \mathbf{I} - \frac{1}{n} \mathbf{1}_{n \times n} \right) \mathbf{K} \left( \mathbf{I} - \frac{1}{n} \mathbf{1}_{n \times n} \right)
\end{align}
where $\mathbf{I}$ is the $n \times n$ identity matrix
and $\mathbf{1}_{n \times n}$ is the $n \times n$ matrix with all elements equal to 1.

#### Normalizing in feature space

One way to normalize points in a feature space is to ensure that they have a unit length by replacing $\phi(\boldsymbol{x}_i)$ with the corresponding unit vector $\phi_n(\boldsymbol{x}_i) = \frac{\phi(\boldsymbol{x}_i)}{\lVert \phi(\boldsymbol{x}_i) \rVert}$.

The dot product in a normalized feature space corresponds to the cosine of the angle between the two mapped points because
$$
\phi_n(\boldsymbol{x}_i)^{\top} \phi_n(\boldsymbol{x}_j)
= \frac{\phi(\boldsymbol{x}_i)^{\top} \phi(\boldsymbol{x}_j)}{\lVert \phi(\boldsymbol{x}_i) \rVert \cdot \lVert \phi(\boldsymbol{x}_j) \rVert}
= \cos(\theta)
$$

If the mapped points are both centered and normalized, then a dot product corresponds to the correlation between the two points in the feature space.

The normalized kernel matrix $\mathbf{K}_n$ can be computed as:
$$
\mathbf{K}_n( \boldsymbol{x}_i, \boldsymbol{x}_j )
= \frac{\phi(\boldsymbol{x}_i)^{\top} \phi(\boldsymbol{x}_j)}{\lVert \phi(\boldsymbol{x}_i) \rVert \cdot \lVert \phi(\boldsymbol{x}_j) \rVert}
= \frac{K(\boldsymbol{x}_i, \boldsymbol{x}_j)}{\sqrt{K(\boldsymbol{x}_i, \boldsymbol{x}_i) \cdot K(\boldsymbol{x}_j, \boldsymbol{x}_j)}}
$$

$\mathbf{K}_n$ has diagonal elements equal to 1.

**Note**: this can be rewritten as:
$$
\mathbf{K}_n = \mathbf{W}^{-1/2} \cdot \mathbf{K} \cdot \mathbf{W}^{-1/2}
$$

with $\mathbf{W}^{-1/2}$ the diagonal matrix defined as
$$
\large
\mathbf{W}^{-1/2} =
\begin{pmatrix}
\frac{1}{\sqrt{K(\boldsymbol{x}_1, \boldsymbol{x}_1)}} & 0                                              & \cdots & 0\\
0                                              & \frac{1}{\sqrt{K(\boldsymbol{x}_2, \boldsymbol{x}_2)}} & \cdots & 0\\
\vdots                                         & \vdots                                         & \ddots & \vdots\\
0                                              & 0                                              & \cdots & \frac{1}{\sqrt{K(\boldsymbol{x}_n, \boldsymbol{x}_n)}}
\end{pmatrix}
$$

## Part 2: Clustering with the kernel k-means algorithm

### Introduction

We have studied the k-means algorithm for clustering tasks [last week](https://adimajo.github.io/CSE204-2021/lab_session_13/lab_session_13.html).
As this algorithm can easily be kernelized, we will use it to make a first implementation of a kernel method: Kernel k-Means.

In k-Means, the separating boundary between clusters is linear.
As we saw in the previous part, the most obvious advantage of applying the *kernel trick* to k-Means is that it allows us to extract nonlinear boundaries between clusters.
In other words, Kernel k-Means can detect nonconvex clusters.

You will see that this *Kernel k-Means* can be seen as an alternative to *Spectral Clustering*, the other clustering algorithm [seen last week](https://adimajo.github.io/CSE204-2021/lab_session_13/lab_session_13.html).

Once again, the main idea here is to conceptually map data points $\boldsymbol{x}$ in the input space to a point $\phi(\boldsymbol{x})$ in some high-dimensional feature space $\mathcal{H}$ via an appropriate nonlinear mapping $\phi$.
The kernel trick allows us to carry out the clustering in feature space without explicitly making computations in this feature space.
All computation will be done in the input space $\mathcal{X}$, using a kernel function $K(\boldsymbol{x}_i, \mathbf{x}_j)$ which will replace every dot (or inner) product $\phi(\boldsymbol{x}_i)^{\top} \phi(\boldsymbol{x}_j)$.
This means that we will have to rewrite the k-means algorithm such that the mapping function $\phi$ only appears in such dot products
$\phi(\boldsymbol{x}_i)^{\top} \phi(\boldsymbol{x}_j)$ (which in the end will be replaced by $K(\boldsymbol{x}_i, \boldsymbol{x}_j)$).

### Notation

We (still) denote by $\boldsymbol{x} \in \mathbb{R}^d$ the point in the input space and
$\phi(\boldsymbol{x})$ its corresponding image in the feature space.
$\mathcal{D} = (\boldsymbol{x}_i)_1^n \subset \mathbb{R}^d$ is our dataset containing $n$ points.
We want to define $k$ clusters $\boldsymbol{\mathcal{C}} = \{\boldsymbol{C}_1, \dots, \boldsymbol{C}_k\}$, each one defining the cluster mean / centroid $\{\boldsymbol{\mu}^{\phi}_1, \dots, \boldsymbol{\mu}^{\phi}_k\}$ in the feature space:
$$
\boldsymbol{\mu}^{\phi}_i = \frac{1}{n_i} \sum_{\boldsymbol{x}_j \in C_i} \phi(\boldsymbol{x}_j)
$$
where $\boldsymbol{\mu}^{\phi}_i$ is the mean of the $i$th cluster $\boldsymbol{C}_i$ in feature space,
with $n_i = |\boldsymbol{C}_i|$.

### Step 1: express the k-means *sum of squared errors* (SSE) objective function in terms of the kernel function

The kernel k-means sum of squared errors (SSE) objective can be written as
$$
SSE(\boldsymbol{\mathcal{C}}) = \sum_{i=1}^k\sum_{\boldsymbol{x}_j \in \boldsymbol{C}_i} \left\lVert \phi(\boldsymbol{x}_j) - \boldsymbol{\mu}^{\phi}_i \right\rVert^2
$$

**Question 1**: show that $SSE(\boldsymbol{\mathcal{C}})$ can be rewritten as:
$$
SSE(\boldsymbol{\mathcal{C}}) =
\left( \sum_{i=1}^k\sum_{\boldsymbol{x}_j \in \boldsymbol{C}_i} \phi(\boldsymbol{x}_j)^{\top} \phi(\boldsymbol{x}_j) \right)
- \left( \sum_{i=1}^k n_i \left\lVert \boldsymbol{\mu}^{\phi}_i \right\rVert^2 \right)
$$

YOUR ANSWER HERE

**Question 2**: show then - using kernel operations seen in Part 1 - that $SSE(\mathcal{C})$ can be rewritten as:
$$
SSE(\boldsymbol{\mathcal{C}}) =
\sum_{j=1}^n K(\boldsymbol{x}_j, \boldsymbol{x}_j)
- \sum_{i=1}^k \frac{1}{n_i}
  \sum_{\boldsymbol{x}_a \in \boldsymbol{C}_i}
  \sum_{\boldsymbol{x}_b \in \boldsymbol{C}_i} K(\boldsymbol{x}_a, \boldsymbol{x}_b)
$$

YOUR ANSWER HERE

As for k-Means, we assign in Kernel k-Means each point to the closest mean / centroid in feature space, resulting in a new clustering, which in turn can be used to obtain new estimates for the cluster means.
The main difficulty is that we cannot explicitly compute the mean of clusters in feature space.
Fortunately, we will see that knowing explicitly the mean of clusters is not required for the clustering task.

### Step 2: express the distance of a point $\phi(\boldsymbol{x}_j)$ to the mean $\boldsymbol{\mu}^{\phi}_i$

**Question 3**: show that
$$
\left\lVert \phi(\boldsymbol{x}_j) - \boldsymbol{\mu}^{\phi}_i \right\rVert^2
=
K(\boldsymbol{x}_j, \boldsymbol{x}_j)
- \frac{2}{n_i} \sum_{\boldsymbol{x}_a \in \boldsymbol{C}_i} K(\boldsymbol{x}_a, \boldsymbol{x}_j)
+ \frac{1}{n^2_i} \sum_{\boldsymbol{x}_a \in \boldsymbol{C}_i}\sum_{\mathbf{x}_b \in \boldsymbol{C}_i} K(\boldsymbol{x}_a, \boldsymbol{x}_b)
$$

YOUR ANSWER HERE

Thanks to this equation, the distance of a point to a cluster mean in feature space can be computed using only kernel operations.

### Step 3: define the cluster assignment

**Question 4**: show that the cluster assignment used at each iteration of the k-means algorithm can be written as follows
$$
\boldsymbol{C}^*(\boldsymbol{x}_j) =
\arg\!\min_i \left\{
\frac{1}{n^2_i} \sum_{\boldsymbol{x}_a \in \boldsymbol{C}_i}\sum_{\boldsymbol{x}_b \in \boldsymbol{C}_i} K(\boldsymbol{x}_a, \boldsymbol{x}_b)
- \frac{2}{n_i} \sum_{\boldsymbol{x}_a \in \boldsymbol{C}_i} K(\boldsymbol{x}_a, \boldsymbol{x}_j)
\right\}.
$$

*Hint:* recall that, in the input space, the cluster assignment is
$$\boldsymbol{C}^*(\boldsymbol{x}_j) =
\arg\!\min_i \left\{
\left\lVert \boldsymbol{x}_j - \boldsymbol{\mu}_i \right\rVert^2
\right\}.
$$

YOUR ANSWER HERE

Note that the first term in the argmin is the average pairwise kernel value for cluster $\boldsymbol{C}_i$ and it is independent of the point $\boldsymbol{x}_j$.
It is the squared norm of the cluster mean in feature space.

The second term is twice the average kernel value for points in $\boldsymbol{C}_i$ with respect to $\boldsymbol{x}_j$.

### Step 4: implement Kernel K-means algorithm

**Question 5**: implement in Python Kernel K-means defined below.

___
### Algorithm 1

Kernel K-means$(\mathbf{K}, k, \epsilon)$

$t \leftarrow 0$ <br>
$\boldsymbol{\mathcal{C}} \leftarrow \{\boldsymbol{C}_1, \dots, \boldsymbol{C}_k\} \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad$ # Randomly partition points into $k$ clusters <br>

**REPEAT**

$\quad t \leftarrow t+1$ <br>

$\quad$**FOREACH** $\boldsymbol{C}_i \in \boldsymbol{\mathcal{C}}^{t-1}$ <br>
$\quad\quad$$\text{sqnorm}_i \leftarrow \frac{1}{n^2_i} \sum_{\boldsymbol{x}_a \in \boldsymbol{C}_i}\sum_{\boldsymbol{x}_b \in \boldsymbol{C}_i} K(\boldsymbol{x}_a, \boldsymbol{x}_b) \quad\quad\quad$ # Compute the squared norm of cluster means <br>

$\quad$**FOREACH** $\boldsymbol{x}_i \in \mathcal{D}$ <br>
$\quad\quad$**FOREACH** $\boldsymbol{C}_i \in \boldsymbol{\mathcal{C}}^{t-1}$ <br>
$\quad\quad\quad$$\text{avg}_{ji} \leftarrow \frac{1}{n_i} \sum_{\boldsymbol{x}_a \in \boldsymbol{C}_i} K(\boldsymbol{x}_a, \boldsymbol{x}_j) \quad\quad\quad\quad\quad\quad$ # Average kernel value for $\boldsymbol{x}_j$ and $\boldsymbol{C}_i$ <br>

$\quad$# Find the closest cluster for each point <br>
$\quad$**FOREACH** $\boldsymbol{x}_i \in \mathcal{D}$ <br>
$\quad\quad$**FOREACH** $\boldsymbol{C}_i \in \mathcal{C}^{t-1}$ <br>
$\quad\quad\quad$$d(\boldsymbol{x}_j, \boldsymbol{C}_i) \leftarrow \text{sqnorm}_i - 2 ~ \text{avg}_{ji}$ <br>
$\quad\quad$$j^* \leftarrow \arg\!\min_i \left\{ d(\boldsymbol{x}_j, \boldsymbol{C}_i) \right\}$ <br>
$\quad\quad$$\boldsymbol{C}^t_{j^*} \leftarrow \boldsymbol{C}^t_{j^*} \cup \{ \boldsymbol{x}_j \} \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad$ # Cluster reassignment <br>

$\quad$$\boldsymbol{\mathcal{C}}^t \leftarrow \{\boldsymbol{C}_1^t, \dots, \boldsymbol{C}_k^t\}$ <br>

**UNTIL** the fraction of points with new cluster assignments $\leq \epsilon$
___

Complete functions `init_points_cluster_index` and `kernel_k_means` below.

In [ ]:
def take(n: int, iterable) -> list:
    "Return first n items of the iterable as a list"
    return list(itertools.islice(iterable, n))

def init_points_cluster_index(n: int, k: int) -> np.ndarray:
    """
    Randomly partition n points into k clusters

    :param int n: number of rows / samples of the resulting array
    :param int k: number of clusters
    :returns: 1D array of length n with random values 0 ... k-1 representing initial cluster values
    :rtype: numpy.ndarray
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test your implementation of init_points_cluster_index
assert init_points_cluster_index(3, 2).shape == (3,)
assert (0 <= init_points_cluster_index(3, 2)).all()
assert (init_points_cluster_index(3, 2) <= 1).all()

In [ ]:
def kernel_k_means(dataset: np.ndarray, K: np.ndarray, k: int, epsilon: float = 0):
    """
    Kernel K-means

    Parameters
    ----------
    dataset : numpy.ndarray
        the dataset to perform kernel k means on
    K : numpy.ndarray
        the n-by-n Kernel matrix of inputs
    k : int
        the number of clusters to find
    epsilon : float
        the termination criterion

    Return
    ------
    points_cluster_index : numpy.ndarray
        a 1D vector of labels of length n (e.g. c[i] = C_j means "x_i is belongs to cluster C_j")
    iterations : int
        the number of iterations until convergence
    loss : float
        the achieved loss
    """

    n = K.shape[0]   # number of  points
    
    if n < k:
        raise Exception("There are less points than clusters")

    # Init points cluster
    points_cluster_index = init_points_cluster_index(n, k)

    finished = False
    iterations = 0
    while not finished:
        if iterations > 100:
            break
        iterations += 1
        previous_points_cluster_index = points_cluster_index

        # Compute "distances" of points to clusters
        # YOUR CODE HERE
        raise NotImplementedError()
            # Number of points in cluster i
            # YOUR CODE HERE
            raise NotImplementedError()
            # Kernel matrix of points belonging to cluster i
            # YOUR CODE HERE
            raise NotImplementedError()
            # Compute the square norm of clusters mean
            # YOUR CODE HERE
            raise NotImplementedError()
            # Kernel matrix of points belonging to cluster i in rows and point j in column
            # Average kernel values between point j and cluster i
            # YOUR CODE HERE
            raise NotImplementedError()
        # Assign clusters to points
        # YOUR CODE HERE
        raise NotImplementedError()
        # YOUR CODE HERE
        raise NotImplementedError()

    return points_cluster_index, iterations, loss

### Step 5: Test the implementation on a set of given datasets

We will use the same datasets than last week:
- the first dataset consists of 4 gaussian-distributed clusters of points with equal variance;
- the second represents two clusters, one stretched vertically, and one horizontally;
- finally, the last dataset represents 3 clusters distributed in rings.

For convenience, the three datasets are placed in a list called `datasets`.

In [ ]:
# Create a data set
N = 120

data1 = np.random.normal((0,0), (0.5, 0.5) ,size=(N,2))
data1 = np.append(data1, np.random.normal((5,0), (0.5, 0.5), size=(N, 2)), axis=0)
data1 = np.append(data1, np.random.normal((0,5), (0.5, 0.5), size=(N, 2)), axis=0)
data1 = np.append(data1, np.random.normal((5,5), (0.5, 0.5), size=(N, 2)), axis=0)

data2 = np.random.normal((2, 5), (0.25, 1), size=(N, 2))
data2 = np.append(data2, np.random.normal((5,5), (1, 0.25), size=(N, 2)), axis=0)

# radii = np.random.normal(0, 0.5, size=(N, 1))
radii = np.random.normal(0, 1.5, size=(N, 1))
# radii = np.append(radii, np.random.normal(4, 0.5, size=(2 * N, 1)), axis=0)
radii = np.append(radii, np.random.normal(8, 0.5, size=(3 * N, 1)), axis=0)
# angles = np.random.uniform(size=(6 * N, 1)) * 2.0 * np.pi
angles = np.random.uniform(size=(4 * N, 1)) * 2.0 * np.pi
data3 = np.hstack([radii * np.cos(angles), radii * np.sin(angles)])

datasets = [data1, data2, data3]

fig, axes = plt.subplots(1, len(datasets), figsize=(10, 3))
for i, data in enumerate(datasets):
    axes[i].scatter(data[:, 0], data[:, 1])

To test your implementation, run the following code which will plot the 3 datasets, trying different values of $k$.

In [ ]:
for kernel in [GaussianKernel(sigma=1.5),
               PolynomialKernel(q=2., c=0.),
               LinearKernel()]:
    print("kernel:", str(kernel))
    fig, axes = plt.subplots(3, len(datasets), figsize=(20, 20))

    for k_index, k in enumerate([2, 3, 4]):
        print("k:", k)
        for dataset_index, data in enumerate(datasets):
            print("dataset:", dataset_index)
            kernel_matrix = make_kernel_matrix(data, kernel)
            clusters, iterations, loss = kernel_k_means(data, kernel_matrix, k, epsilon=0)
            axes[k_index, dataset_index].scatter(data[:,0], data[:,1], c=clusters, cmap='rainbow')
            axes[k_index, dataset_index].set_title('{} kernel k={}  iter={}  loss={:0.1f}'.format(str(kernel), k, iterations, loss))

    fig.show()

## Part 3: Feature extraction with the Kernel PCA algorithm

We have studied the Principal Component Analysis (PCA) algorithm for features extraction tasks in [`lab_session_10`](https://adimajo.github.io/CSE204-2021/lab_session_10/lab_session_10.html).
PCA can be "kernelized" to find nonlinear "directions" in the data.

Kernel PCA finds the directions of most variance in the feature space instead of the input space.

Again, using the *kernel trick*, all operations can be carried out in terms of the kernel function in the input space, without having to transform the data into feature space.

We won't detail the kernelization process here but instead we will focus on the implementation and the obtained results.

As a reminder, the PCA algorithm is described in algorithm 2 defined below. Its kernelized version is defined in algorithm 3.

___
### Algorithm 2

PCA$(\mathbf{D}, r)$
$$
\begin{array}{lrcl}
{\tiny 1.} \quad\quad & \boldsymbol{\mu}    & = & \frac{1}{n} \sum_{i=1}^{n} \boldsymbol{x}_i                                          & \text{# Compute the mean} \\
{\tiny 2.}            & \mathbf{Z}      & = & \mathbf{D} - \mathbf{1}_{n \times d} \cdot \boldsymbol{\mu}^{\top}                                & \text{# Center the dataset } \mathbf{D} \text{ (c.f. note below)} \\
{\tiny 3.}            & \mathbf{\Sigma} & = & \frac{1}{n} \left( \mathbf{Z}^{\top} \mathbf{Z} \right)                          & \text{# Compute the covariance matrix} \\
{\tiny 4.}          &  \boldsymbol{\lambda} =  (\lambda_1, \lambda_2, \dots, \lambda_d)                         & = & \text{eigenvalues}(\mathbf{\Sigma})  & \text{# Compute eigenvalues} \\
{\tiny 5.}            & \mathbf{U}   = \pmatrix{ \boldsymbol{u}_1 & \boldsymbol{u}_2 & \dots & \boldsymbol{u}_d } & = & \text{eigenvectors}(\mathbf{\Sigma}) & \text{# Compute eigenvectors} \\
{\tiny 6.}            & \mathbf{U}_r & = & \pmatrix{\boldsymbol{u}_1 & \boldsymbol{u}_2 & \dots & \boldsymbol{u}_r}                                   & \text{# Reduced basis} \\
{\tiny 7.}            & \mathbf{A}   & = & \left\{\boldsymbol{a}_i | \boldsymbol{a}_i = \mathbf{U}_r^{\top}\boldsymbol{x}_i ~ \text{ for } i = 1, \dots, n \right\} \quad  & \text{# Projected data} \\
\end{array}
$$

___


Here, $\mathbf{D}$ is the $n \times d$ dataset (or "design") matrix ($n$ points of $d$ dimensions) and $r$ is the size of the basis we want to compute.

$\mathbf{1}_{n \times d}$ is the $n \times d$ matrix filled with 1s ([numpy.ones(shape=(n, d))](https://numpy.org/doc/stable/reference/generated/numpy.ones.html?highlight=ones#numpy.ones) in Python).

**Note:** The $\mathbf{1} \cdot \boldsymbol{\mu}^{\top}$ matrix can be made with [np.tile(mu, (d, 1))](https://numpy.org/doc/stable/reference/generated/numpy.tile.html?highlight=tiles) in Python (assuming $\boldsymbol{\mu}$ is a 1D numpy array).

**Question 1**: implement in Python the Kernel PCA algorithm defined below.

___
### Algorithm 3

Kernel PCA$(\mathbf{D}, K, r)$

$$
\begin{array}{lrcl}
{\tiny 1.} \quad\quad & \mathbf{K}   & = & \{ K(\boldsymbol{x}_i, \boldsymbol{x}_j) \}_{i,j=1, \dots, n}                                                              & \text{# Compute the } n \times n \text{ kernel matrix from the dataset } \mathbf{D} \\
{\tiny 2.}            & \mathbf{K}   & = & (\mathbf{I} - \frac{1}{n}\mathbf{1}_{n \times n}) ~ \mathbf{K} ~ (\mathbf{I} - \frac{1}{n}\mathbf{1}_{n \times n}) & \text{# Center the kernel matrix (c.f. Part 1)} \\
{\tiny 3.}            & (\eta_1, \eta_2, \dots, \eta_d)                   & = & \text{eigenvalues}(\mathbf{K})                                                & \text{# Compute eigenvalues} \\
{\tiny 4.}            & \pmatrix{ \boldsymbol{c}_1 & \boldsymbol{c}_2 & \dots & \boldsymbol{c}_n } & = & \text{eigenvectors}(\mathbf{K})                                               & \text{# Compute eigenvectors} \\
{\tiny 5.}            & \mathbf{C}_r & = & \pmatrix{ \boldsymbol{c}_1 & \boldsymbol{c}_2 & \dots & \boldsymbol{c}_r }                                                                  & \text{# Reduced basis} \\
{\tiny 6.}            & \mathbf{A}   & = & \left\{ \boldsymbol{a}_i | \boldsymbol{a}_i = \mathbf{C}_r^{\top}\mathbf{K}_i ~ \text{ for } i = 1, \dots, n \right\}  \quad             & \text{# Projected data} \\
\end{array}
$$

___


Here, $\mathbf{D}$ is the $n \times d$ dataset (or "design") matrix ($n$ points of $d$ dimensions), $K$ is a kernel function and $r$ is the size of the basis we want to compute.

$\mathbf{I}$ is the $n \times n$ identity matrix
and $\mathbf{1}_{n \times n}$ is the $n \times n$ matrix filled with 1s ([numpy.ones(shape=(n, n))](https://numpy.org/doc/stable/reference/generated/numpy.ones.html?highlight=ones#numpy.ones) in Python).

**Note**: Eigenvectors and eigenvalues can be computed with [numpy.linalg.eig(K)](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html#numpy.linalg.eig).

In [ ]:
def kernel_pca(data: np.ndarray, kernel_function, basis: int):
    """
    Kernel PCA algorithm

    Parameters
    ----------
    data : numpy.ndarray
        the n-by-d matrix of inputs (n points of d dimensions - D in the equations above)
    kernel_function : function
        the kernel function used to make the kernel matrix of inputs (K in the equations above)
    basis : int
        the number of basis to compute (r in the equations above)

    Return
    ------
    projected_data : numpy.ndarray
        a 1D vector of labels of length n (e.g. c[i] = C_j means "x_i is belongs to cluster C_j")
    """

    n = data.shape[0]   # number of  points

    # Compute the kernel matrix
    # YOUR CODE HERE
    raise NotImplementedError()

    # Center the kernel matrix
    # YOUR CODE HERE
    raise NotImplementedError()

    # Compute eigenvalues and eigenvectors
    # YOUR CODE HERE
    raise NotImplementedError()

    # Reduced basis
    # YOUR CODE HERE
    raise NotImplementedError()
   
    # Projected data
    # YOUR CODE HERE
    raise NotImplementedError()
    return projected_data

**Question 2**: test the implementation on the following dataset (using the quadratic kernel defined in Part 1).

In [ ]:
np.random.seed(123)
data = np.random.multivariate_normal(mean=np.zeros(2), cov=np.array([[1, 0], [0, 1]]), size=100)
data[:,0] = 0.2 * data[:,0]**2 + data[:,1]**2 + 0.1 * data[:,0] * data[:,1]
plt.scatter(data[:,0], data[:,1]);

In [ ]:
kernel = PolynomialKernel()
projected_data = kernel_pca(data, kernel, 2)

In [ ]:
plt.scatter(projected_data[:, 0], projected_data[:, 1])
plt.show();

## Part 4: Kernel Ridge Regression (Bonus)

**Question 1**: implement in Python the Kernel Ridge Regression algorithm defined in [the lecture notes](https://moodle.polytechnique.fr/pluginfile.php/203685/mod_resource/content/1/Notes_12.pdf).

In [ ]:
def kernel_ridge_regression(x: float, dataset: np.ndarray, kernel_function, lambda_: float):
    """
    Kernel ridge regression
    
    Parameters
    ----------
    x : float
        the value to predict
    dataset : ndarray
        the dataset used to make the kernel matrix of inputs
    kernel_function : function
        the kernel function used to make the kernel matrix of inputs
    lambda_ : float
        the regularization strength (lambda is a reserved python keyword)

    Return
    ------
    y_pred : float
        the predicted value for x
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return y_pred

**Question 2**: test the implementation on the following non linear 1D dataset used in lab 4.

In [ ]:
dataset = pd.DataFrame([[2., 0.],
                        [5., 2.],
                        [7., 1.],
                        [10., 2.],
                        [14., 4.],
                        [16., 3.],
                        [17., 0.]], columns=['x', 'y'])
dataset

In [ ]:
kernel_function = PolynomialKernel()

In [ ]:
x_pred = np.linspace(0., 20., 200)
y_pred = [kernel_ridge_regression(x, dataset, kernel_function, lambda_=6.) for x in x_pred]

ax = dataset.plot.scatter(x='x', y='y', label="Dataset", figsize=(12,8))
ax.plot(x_pred, y_pred, "-r", label="Kernel ridge regression")
plt.legend();